In [ ]:
from dataclasses import dataclass
from ortools.sat.python import cp_model
import opt_note.scsp as scsp

In [ ]:
import marimo as mo
import nbformat

# 順序制約付き巡回セールスマン問題として定式化する

In [ ]:
@dataclass
class Model:
    instance: list[str]
    solution: str | None = None
    best_bound: float = 0.0

    def solve(
        self, time_limit: int | None = 60, log: bool = False, *args, **kwargs
    ) -> str | None:
        cpmodel = cp_model.CpModel()
        cpsolver = cp_model.CpSolver()

        nodes = [
            (sidx, cidx)
            for sidx, s in enumerate(self.instance)
            for cidx, _ in enumerate(s)
        ]
        order = [cpmodel.new_int_var(1, len(nodes), "") for _ in nodes]

        dummy_idx = len(nodes)
        order.append(cpmodel.new_constant(0))

        arcs = []
        costs = dict()

        for nidx, (sidx, cidx) in enumerate(nodes):
            if cidx == 0:
                arcs.append((dummy_idx, nidx, cpmodel.new_bool_var("")))
                costs[(dummy_idx, nidx)] = 1
            if cidx == len(self.instance[sidx]) - 1:
                arcs.append((nidx, dummy_idx, cpmodel.new_bool_var("")))
                costs[(nidx, dummy_idx)] = 0

        for nidx1, (sidx1, cidx1) in enumerate(nodes):
            for nidx2, (sidx2, cidx2) in enumerate(nodes):
                if sidx1 == sidx2 and cidx1 + 1 != cidx2:
                    continue
                s1 = self.instance[sidx1]
                s2 = self.instance[sidx2]
                arcs.append((nidx1, nidx2, cpmodel.new_bool_var("")))
                costs[(nidx1, nidx2)] = (
                    0 if sidx1 < sidx2 and s1[cidx1] == s2[cidx2] else 1
                )

        cpmodel.add_circuit(arcs)

        for nidx1, nidx2, v in arcs:
            if nidx2 == dummy_idx:
                continue
            cpmodel.add(order[nidx2] == order[nidx1] + 1).only_enforce_if(v)

        nidx = -1
        for s in self.instance:
            for cidx, _ in enumerate(s):
                nidx += 1
                if cidx == 0:
                    continue
                cpmodel.add(order[nidx - 1] < order[nidx])

        cpmodel.minimize(sum(costs[(nidx1, nidx2)] * v for (nidx1, nidx2, v) in arcs))

        cpsolver.parameters.log_search_progress = log
        if time_limit is not None:
            cpsolver.parameters.max_time_in_seconds = time_limit
        status = cpsolver.solve(cpmodel)

        self.best_bound = cpsolver.best_objective_bound

        if status in {
            cp_model.cp_model_pb2.OPTIMAL,
            cp_model.cp_model_pb2.FEASIBLE,
        }:
            solution = ""
            current_node = dummy_idx
            current_char: str | None = None
            current_sidxs: set[int] = set()
            complete = False
            while True:
                for nidx1, nidx2, v in arcs:
                    if nidx1 == current_node and cpsolver.boolean_value(v):
                        if nidx2 == dummy_idx:
                            complete = True
                            break
                        sidx, cidx = nodes[nidx2]

                        if self.instance[sidx][cidx] != current_char or sidx in current_sidxs:
                            solution += self.instance[sidx][cidx]
                            current_sidxs.clear()

                        current_node = nidx2
                        current_char = self.instance[sidx][cidx]
                        current_sidxs.add(sidx)
                if complete:
                    break
            self.solution = solution
        else:
            self.solution = None

        return self.solution

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n004k015-025.txt", log=True)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x99bbd015ce8d5085)
#Variables: 5'433 (#bools: 5'241 in objective) (5'433 primary variables)
  - 5'348 Booleans in [0,1]
  - 84 in [1,84]
  - 1 constants in {0} 
#kCircuit: 1
#kLinear2: 5'424 (#enforced: 5'344)

Starting presolve at 0.00s
  1.15e-03s  0.00e+00d  [DetectDominanceRelations] 
  1.01e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  2.02e-05s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.91e-04s  0.00e+00d  [DetectDuplicateColumns] 
  8.01e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 16'373 nodes and 32'315 arcs.
[Symmetry] Symmetry computation done. time: 0.00102516 dtime: 0.00246339
  9.38e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.59e-01s  1.63e-01d  [Probe] #probed=10'704 #new_binary_clauses=3'

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n008k015-025.txt", log=True)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x6d8144a6890fd4cf)
#Variables: 27'109 (#bools: 26'353 in objective) (27'109 primary variables)
  - 26'933 Booleans in [0,1]
  - 175 in [1,175]
  - 1 constants in {0} 
#kCircuit: 1
#kLinear2: 27'092 (#enforced: 26'925)

Starting presolve at 0.01s
  5.78e-03s  0.00e+00d  [DetectDominanceRelations] 
  5.27e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  1.33e-04s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.21e-03s  0.00e+00d  [DetectDuplicateColumns] 
  4.69e-03s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 81'484 nodes and 162'074 arcs.
[Symmetry] Symmetry computation done. time: 0.0051507 dtime: 0.0123967
  5.32e-03s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  3.04e+00s  1.00e+00d *[Probe] #probed=28'888 #new_binary_cla

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n016k015-025.txt", log=True)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x5107eee0b9c6d108)
#Variables: 98'291 (#bools: 95'995 in objective) (98'291 primary variables)
  - 97'967 Booleans in [0,1]
  - 323 in [1,323]
  - 1 constants in {0} 
#kCircuit: 1
#kLinear2: 98'258 (#enforced: 97'951)

Starting presolve at 0.03s
  2.17e-02s  0.00e+00d  [DetectDominanceRelations] 
  1.90e+00s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  1.16e-03s  0.00e+00d  [ExtractEncodingFromLinear] 
  5.10e-03s  0.00e+00d  [DetectDuplicateColumns] 
  2.05e-02s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 295'162 nodes and 588'674 arcs.
[Symmetry] Symmetry computation done. time: 0.0210727 dtime: 0.0450922
  2.03e-02s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  3.45e+00s  1.00e+00d *[Probe] #probed=15'272 #new_binary_cl

う～ん非常に良くない.
なんならただの線形計画問題としての定式化 `LINEAR_CPSAT` よりも悪い.